# La Vuelta a Espana (2001-2022)

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd

def fetch_url(url):
    '''
    Returns the content of the `url`.
    '''
    r = requests.get(url)
    if r.status_code == 200:
        return r.content
    raise Exception('Request Error')

## Scrape Stage Data

In [2]:
base_url = 'https://www.procyclingstats.com/race/vuelta-a-espana/{}/stage-{}'
# Initialize data structures. 
# Results contains individual stages, including time trials. TTT are team time trials. Info has details about the stages.
results = pd.DataFrame()
ttt = pd.DataFrame()
info = []

for year in range(2001, 2023):
    for stage in range(1, 22):
        url = base_url.format(str(year), str(stage))
        # Access url with BeautifulSoup.
        stats = bs(fetch_url(url), 'html.parser')
        
        # Info
        infolist = []
        ul = stats.find('ul', class_ = 'infolist')
        # There are not always 21 stages.
        if ul is None:
            break
            
        for index, li in enumerate(ul.find_all('li', recursive = False)[:-1]):
            if index == 7:
                # Parcours Type
                infolist.append(li.find_all('span')[0]['class'])
            else:
                infolist.append(li.find_all('div', recursive = False)[-1].text)
                
        infolist.append(stage)
        info.append(infolist)
        
        # Stage Results.
        description = stats.find('option', value = url[32:]).text
        
        if 'TTT' in description:
            standings = stats.find_all('table', class_ = 'results-ttt')[0]
            standings = pd.read_html(str(standings), flavor = 'bs4', header = [0])[0]
            standings['year'] = year
            standings['stage'] = stage
        
            ttt = pd.concat([ttt, standings], axis = 0)
            
        else:
            standings = stats.find_all('table', class_ = 'results basic moblist10')[0]
            standings = pd.read_html(str(standings), flavor = 'bs4', header = [0])[0]
            
            if 'ITT' in description:
                standings['type'] = 'ITT'
            elif 'prologue' in description.lower():
                standings['type'] = 'Prologue'
            else:
                standings['type'] = 'Road Race'
                
            standings['year'] = year
            standings['stage'] = stage
        
            results = pd.concat([results, standings], axis = 0)

In [3]:
ttt.to_csv('ttt_raw', index = False)
results.to_csv('results_raw', index = False)

## Clean `info`

In [4]:
info_cols = ['Date', 'Start', 'Speed', 'Category', 'Distance', 'Points_Scale', 'UCI_Scale', 'Parcour_Type', 'Profile_Score',
             'Vertical_Meters', 'Departure', 'Arrival', 'Race_Ranking', 'List_Quality', 'Won_How', 'stage']
info_df = pd.DataFrame(info, columns = info_cols)
info_df.to_csv('info_raw', index = False)
info_df.tail()

,Date,Start,Speed,Category,Distance,Points_Scale,UCI_Scale,Parcour_Type,Profile_Score,Vertical_Meters,Departure,Arrival,Race_Ranking,List_Quality,Won_How,stage
454,07 September 2022,13:21,43.773 km/h,ME - Men Elite,162.3 km,GT.B.Stage,UCI.WR.GT.B.Stage - TM2022,"[icon, profile, p1]",161,2755,Aracena,Monasterio de Tentudía,9,970,Sprint of small group,17
455,08 September 2022,12:19,40.381 km/h,ME - Men Elite,192 km,GT.B.Stage,UCI.WR.GT.B.Stage - TM2022,"[icon, profile, p5]",214,3680,Trujillo,Alto del Piornal,9,970,Sprint of small group,18
456,09 September 2022,14:03,41.66 km/h,ME - Men Elite,138.3 km,GT.B.Stage,UCI.WR.GT.B.Stage - TM2022,"[icon, profile, p4]",101,2278,Talavera de la Reina,Talavera de la Reina,9,970,Sprint of large group,19
457,10 September 2022,12:45,38.57 km/h,ME - Men Elite,181 km,GT.B.Stage,UCI.WR.GT.B.Stage - TM2022,"[icon, profile, p4]",216,3977,Moralzarzal,Puerto de Navacerrada,9,970,7.2 km solo,20
458,11 September 2022,17:26,39.577 km/h,ME - Men Elite,96.7 km,GT.B.Stage,UCI.WR.GT.B.Stage - TM2022,"[icon, profile, p1]",10,773,Las Rozas,Madrid,9,970,Sprint of large group,21


In [5]:
info_df.replace({'':np.nan}, inplace = True)

In [6]:
(info_df['Start'] == '-').sum()

316

In [7]:
info_df['Start'] = info_df['Start'].replace({'-':np.nan})

In [8]:
info_df['day'] = info_df['Date'].str[:2].astype(int)
info_df['Month'] = info_df['Date'].str.extract('([a-zA-z]+)')
info_df['year'] = info_df['Date'].str[-4:].astype(int)
info_df.drop('Date', axis = 1, inplace = True)

In [9]:
# All speeds in km/h.
assert info_df['Speed'].str[-4:].nunique() == 1

In [10]:
info_df['Speed'] = info_df['Speed'].str[:-4].astype(float)

In [11]:
info_df['Category'].value_counts()

ME - Men Elite    459
Name: Category, dtype: int64

In [12]:
info_df.drop('Category', axis = 1, inplace = True)

In [13]:
# All distances in km.
assert info_df['Distance'].str[-2:].nunique() == 1

In [14]:
info_df['Distance'] = info_df['Distance'].str[:-2].astype(float)

In [15]:
info_df.drop(['Points_Scale', 'UCI_Scale'], axis = 1, inplace = True)

In [16]:
info_df['Parcour_Type'].value_counts().sort_index()

[icon, profile, p1]    155
[icon, profile, p2]     96
[icon, profile, p3]     48
[icon, profile, p4]     32
[icon, profile, p5]    128
Name: Parcour_Type, dtype: int64

In [17]:
info_df['Parcour_Type'] = info_df['Parcour_Type'].str[-1]

In [18]:
info_df['Race_Ranking'].value_counts(dropna = False)

0     399
10     21
9      21
14     18
Name: Race_Ranking, dtype: int64

In [19]:
info_df.drop('Race_Ranking', axis = 1, inplace = True)

In [20]:
info_df['Won_How'].unique()

array(['Time Trial', 'Sprint of large group', 'Sprint of small group',
       '? km solo', 'Sprint a deux', '35 km solo', '40 km solo',
       '10 km solo', '2 km solo', '0.8 km solo', '4 km solo',
       '25 km solo', '5 km solo', '44 km solo', '51 km solo',
       '15 km solo', '12 km solo', '1 km solo', '20 km solo',
       '16 km solo', '0.5 km solo', '0.4 km solo', '4.3 km solo',
       '162 km solo', '7 km solo', '24.5 km solo', '2.5 km solo',
       '7.5 km solo', '105 km solo', '8 km solo', '11 km solo',
       '26 km solo', '9 km solo', '22 km solo', '3.5 km solo',
       '3 km solo', '0.7 km solo', '5.5 km solo', '28 km solo',
       '22.3 km solo', '0.6 km solo', '4.8 km solo', '1.1 km solo',
       '6 km solo', '13 km solo', '21 km solo', '11.3 km solo',
       '13.6 km solo', '4.6 km solo', '1.5 km solo', '47 km solo',
       '24 km solo', '43 km solo', '6.5 km solo', '1.8 km solo',
       '5.3 km solo', '8.8 km solo', '29 km solo', '0.9 km solo',
       '114 km solo', '1.

In [21]:
info_df.head()

,Start,Speed,Distance,Parcour_Type,Profile_Score,Vertical_Meters,Departure,Arrival,List_Quality,Won_How,stage,day,Month,year
0,NaN,49.420,12.3,p1,2,95,Salamanca,Salamanca,927,Time Trial,1,8,September,2001
1,NaN,41.181,147.2,p1,12,673,Salamanca,Valladolid,927,Sprint of large group,2,9,September,2001
2,NaN,46.108,140.5,p1,3,567,Valladolid,León,927,Sprint of large group,3,10,September,2001
3,NaN,42.039,175.0,p2,50,1942,León,Gijón,927,Sprint of small group,4,11,September,2001
4,NaN,37.306,160.8,p5,282,2946,Gijón,Lagos de Covadonga,927,? km solo,5,12,September,2001


## Clean `ttt` 

In [22]:
ttt.reset_index(drop = True, inplace = True)

In [23]:
ttt.head(50)

,Pos.,Team,Time,Timegap,Speed,PCS points,UCI points,year,stage
0,1.0,O.N.C.E. - Eroski,26:21,NaN,55.787,NaN,NaN,2002,1
1,2.0,US Postal Service,26:35,0:14,55.298,NaN,NaN,2002,1
2,3.0,Kelme - Costa Blanca,26:36,0:15,55.263,NaN,NaN,2002,1
3,4.0,Team Telekom,26:39,0:18,55.159,NaN,NaN,2002,1
4,5.0,Fassa Bortolo,26:44,0:23,54.988,NaN,NaN,2002,1
5,6.0,Cofidis,26:47,0:26,54.885,NaN,NaN,2002,1
6,7.0,iBanesto.com,26:48,0:27,54.851,NaN,NaN,2002,1
7,8.0,Team Coast,27:00,0:39,54.444,NaN,NaN,2002,1
8,9.0,Mapei - Quickstep,27:03,0:42,54.344,NaN,NaN,2002,1
9,10.0,Acqua & Sapone,27:04,0:43,54.310,NaN,NaN,2002,1


In [24]:
ttt.shape

(2971, 9)

In [25]:
ttt.drop(['PCS points', 'UCI points'], axis = 1, inplace = True)

In [26]:
ttt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Pos.     326 non-null    float64
 1   Team     2971 non-null   object 
 2   Time     326 non-null    object 
 3   Timegap  311 non-null    object 
 4   Speed    326 non-null    float64
 5   year     2971 non-null   int64  
 6   stage    2971 non-null   int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 162.6+ KB


In [27]:
import math

# Create a dataframe with the rider names of each team.
riders = []
for i, row in ttt.iterrows():
    if math.isnan(row['Pos.']):
        # Null position indicate rider row.
        names.append(row['Team'])
    else:
        # Non-null posotion is a team entry.
        if i != 0:
            data.append(names)
            riders.append(data)
        data = [row['Team'], row['year'], row['stage']]
        names = []
        
data.append(names)
riders.append(data)
riders = pd.DataFrame(riders, columns = ['Team', 'year', 'stage', 'riders'])
riders.tail()

,Team,year,stage,riders
321,Intermarché - Wanty - Gobert Matériaux,2022,1,"[TAARAMÄE Rein, HIRT Jan, POZZOVIVO Domenico, ..."
322,Euskaltel - Euskadi,2022,1,"[MATÉ Luis Ángel +0:17, MARTÍN Gotzon, BOU Joa..."
323,Lotto Soudal,2022,1,"[BEULLENS Cedric, DRIZNERS Jarrad, VAN GILS Ma..."
324,Cofidis,2022,1,"[VILLELLA Davide, CHAMPION Thomas +0:59, FERNÁ..."
325,Burgos-BH,2022,1,"[NAVARRO Daniel, BOL Jetse, EZQUERRA Jesús, LA..."


In [28]:
# Filter `ttt` dataframe to have only teams in rows, and add team members information.
ttt = ttt[ttt[['Pos.']].notnull().all(1)]
merge_cols = ['Team', 'year', 'stage']
ttt = ttt.merge(riders, how = 'left', left_on = merge_cols, right_on = merge_cols)
ttt.tail()

,Pos.,Team,Time,Timegap,Speed,year,stage,riders
321,19.0,Intermarché - Wanty - Gobert Matériaux,26:05,1:25,53.597,2022,1,"[TAARAMÄE Rein, HIRT Jan, POZZOVIVO Domenico, ..."
322,20.0,Euskaltel - Euskadi,26:12,1:32,53.359,2022,1,"[MATÉ Luis Ángel +0:17, MARTÍN Gotzon, BOU Joa..."
323,21.0,Lotto Soudal,26:29,1:49,52.788,2022,1,"[BEULLENS Cedric, DRIZNERS Jarrad, VAN GILS Ma..."
324,22.0,Cofidis,26:30,1:50,52.755,2022,1,"[VILLELLA Davide, CHAMPION Thomas +0:59, FERNÁ..."
325,23.0,Burgos-BH,26:38,1:58,52.491,2022,1,"[NAVARRO Daniel, BOL Jetse, EZQUERRA Jesús, LA..."


In [29]:
ttt.isnull().sum()[ttt.isnull().sum() > 0]

Timegap    15
dtype: int64

In [30]:
# Timegap is null for teams that rank first.
ttt['Timegap'].fillna('0:00', inplace = True)

## Clean `results`

In [31]:
results = pd.read_csv('results_raw', low_memory = False)

In [32]:
results.tail()

,Rnk,GC,Timelag,BIB,H2H,Rider,Age,Team,UCI,Pnt,Unnamed: 10,Time,Avg,type,year,stage
75981,130,110.0,+4:33:12,156,NaN,HVIDEBERG Jonas IversbyTeam DSM,23,Team DSM,NaN,NaN,NaN,4:444:44,NaN,Road Race,2022,21
75982,131,117.0,+4:48:07,162,NaN,CATALDO DarioTrek - Segafredo,37,Trek - Segafredo,NaN,NaN,NaN,",,4:44",NaN,Road Race,2022,21
75983,132,107.0,+4:23:20,183,NaN,JANSSENS JimmyAlpecin-Deceuninck,33,Alpecin-Deceuninck,NaN,NaN,NaN,4:464:46,NaN,Road Race,2022,21
75984,133,94.0,+3:59:57,212,NaN,AZPARREN Xabier MikelEuskaltel - Euskadi,23,Euskaltel - Euskadi,NaN,NaN,NaN,5:305:30,NaN,Road Race,2022,21
75985,134,97.0,+4:03:39,166,NaN,LÓPEZ Juan PedroTrek - Segafredo,25,Trek - Segafredo,NaN,NaN,NaN,5:455:45,NaN,Road Race,2022,21


In [33]:
results.reset_index(drop = True, inplace = True)

In [34]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75986 entries, 0 to 75985
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rnk          75986 non-null  object 
 1   GC           73757 non-null  float64
 2   Timelag      73757 non-null  object 
 3   BIB          75986 non-null  int64  
 4   H2H          0 non-null      float64
 5   Rider        75986 non-null  object 
 6   Age          75986 non-null  int64  
 7   Team         75889 non-null  object 
 8   UCI          1264 non-null   float64
 9   Pnt          6651 non-null   float64
 10  Unnamed: 10  204 non-null    object 
 11  Time         75986 non-null  object 
 12  Avg          6587 non-null   float64
 13  type         75986 non-null  object 
 14  year         75986 non-null  int64  
 15  stage        75986 non-null  int64  
dtypes: float64(5), int64(4), object(7)
memory usage: 9.3+ MB


In [35]:
out = ['DNF', 'DNS', 'OTL', 'DSQ']
for col in ['GC', 'Timelag']:
    # No GC or Timelag for riders who do not finish the stage.
    results[col] = results.apply(lambda row: '-' if row['Rnk'] in out else row[col], axis = 1)

In [36]:
results.drop(['H2H', 'UCI', 'Pnt', 'Avg'], axis = 1, inplace = True)

In [37]:
# Rider Column has the Team column appended to it. Let's check if those rows + the ones with null teams add to the total.
team_in_rider = results.apply(lambda x: str(x.Team) in str(x.Rider), axis = 1)
assert sum(team_in_rider) + results['Team'].isnull().sum() == results.shape[0]

In [38]:
# Remove Team name from Rider column.
results['Rider'] = results.apply(lambda x: str(x.Rider).removesuffix(str(x.Team)), axis = 1)

In [39]:
results[results['Unnamed: 10'].notnull()]

,Rnk,GC,Timelag,BIB,Rider,Age,Team,Unnamed: 10,Time,type,year,stage
69468,1,136.0,+2:07,26,PHILIPSEN Jasper,23,Alpecin-Fenix,10″,3:58:5710″,Road Race,2021,2
69469,2,48.0,+0:45,91,JAKOBSEN Fabio,24,Deceuninck - Quick Step,9″,",,0:009″",Road Race,2021,2
69470,3,3.0,+0:10,181,MATTHEWS Michael,30,Team BikeExchange,4″,",,0:004″",Road Race,2021,2
69472,5,2.0,+0:04,32,ARANBURU Alex,25,Astana - Premier Tech,2″,",,0:002″",Road Race,2021,2
69506,39,50.0,+0:45,97,VAN LERBERGHE Bert,28,Deceuninck - Quick Step,1″,",,0:001″",Road Race,2021,2
...,...,...,...,...,...,...,...,...,...,...,...,...
75853,2,102.0,+4:14:48,167,PEDERSEN Mads,26,Trek - Segafredo,6″,",,0:006″",Road Race,2022,21
75854,3,111.0,+4:33:23,172,ACKERMANN Pascal,28,UAE Team Emirates,4″,",,0:004″",Road Race,2022,21
75883,32,2.0,+2:02,124,MAS Enric,27,Movistar Team,3″,",,0:113″",Road Race,2022,21
75956,105,14.0,+29:19,81,CARAPAZ Richard,29,INEOS Grenadiers,1″,",,1:131″",Road Race,2022,21


In [40]:
# The `Unnamed: 10` column represents time bonifications.
results.rename({'Unnamed: 10':'Bonus'}, axis = 1, inplace = True)
results['Bonus'].str[-1].value_counts()

″    204
Name: Bonus, dtype: int64

In [41]:
# As happened with riders and teams, the `Time` column has Bonus appended to it.
bonus_non_null = results.query('Bonus.notnull()')
bonus_at_time = results.apply(lambda x: str(x['Time']).endswith(str(x['Bonus'])), axis = 1)
assert sum(bonus_at_time) == bonus_non_null.shape[0]

In [42]:
# Delete bonification from time.
results['Time'] = results.apply(lambda x: str(x['Time']).removesuffix(str(x['Bonus'])), axis = 1).astype(str)

In [43]:
# Null bonus means zero time benefit. Delete the apostrophe at the end.
results['Bonus'] = results['Bonus'].fillna('00').str[:-1].astype(int)

In [44]:
results.iloc[:15]['Time']

0      0:14:56
1     0:010:01
2     0:060:06
3     0:170:17
4     0:230:23
5     0:240:24
6     0:250:25
7     0:260:26
8       ,,0:26
9     0:280:28
10    0:300:30
11    0:310:31
12    0:320:32
13    0:330:33
14      ,,0:33
Name: Time, dtype: object

In [45]:
# Two observations from above: 
# Equal times as previous rider start with a ','. Some times are duplicated.
double_time = results[results.apply(lambda x: not x['Time'].startswith(',') 
                                    and x['Time'] != '-' 
                                    and x['Rnk'] != '1', axis = 1)]
double_time.head()

,Rnk,GC,Timelag,BIB,Rider,Age,Team,Bonus,Time,type,year,stage
1,2,2.0,+0:01,102,BOTERO Santiago,28,Kelme - Costa Blanca,0,0:010:01,ITT,2001,1
2,3,3.0,+0:06,6,LEIPHEIMER Levi,27,US Postal Service,0,0:060:06,ITT,2001,1
3,4,4.0,+0:17,155,GONZÁLEZ DE GALDEANO Igor,27,O.N.C.E. - Eroski,0,0:170:17,ITT,2001,1
4,5,5.0,+0:23,152,BELOKI Joseba,28,O.N.C.E. - Eroski,0,0:230:23,ITT,2001,1
5,6,6.0,+0:24,86,PLAZA David,31,Festina,0,0:240:24,ITT,2001,1


In [46]:
# Confirm all specified times are doubled.
suffix_num = double_time[double_time['Time'].map(
    lambda t: t[:int(len(t) / 2)] != t[int(len(t) / 2):])][['Time', 'Bonus']]
assert suffix_num.shape[0] == 0

In [47]:
results[results['Rnk'] == '1']

,Rnk,GC,Timelag,BIB,Rider,Age,Team,Bonus,Time,type,year,stage
0,1,1.0,+0:00,41,MILLAR David,24,Cofidis,0,0:14:56,ITT,2001,1
189,1,118.0,+1:40,201,ZABEL Erik,31,Team Telekom,0,3:34:28,Road Race,2001,2
378,1,79.0,+1:40,201,ZABEL Erik,31,Team Telekom,0,3:02:50,Road Race,2001,3
566,1,43.0,+1:23,201,ZABEL Erik,31,Team Telekom,0,4:09:46,Road Race,2001,4
753,1,2.0,+0:15,29,MERCADO Juan Miguel,23,iBanesto.com,0,4:18:37,Road Race,2001,5
...,...,...,...,...,...,...,...,...,...,...,...,...
75304,1,9.0,+9:33,61,URÁN Rigoberto,35,EF Education-EasyPost,22,3:42:28,Road Race,2022,17
75446,1,1.0,+0:00,134,EVENEPOEL Remco,22,Quick-Step Alpha Vinyl Team,10,4:45:17,Road Race,2022,18
75584,1,102.0,+3:43:22,167,PEDERSEN Mads,26,Trek - Segafredo,10,3:19:11,Road Race,2022,19
75718,1,14.0,+28:18,81,CARAPAZ Richard,29,INEOS Grenadiers,13,4:41:34,Road Race,2022,20


In [48]:
def clean_time(row):
    if row['Time'].startswith(','):
        if row['Rnk'] == '2':
            # Same time as first rank.
            return '+0:00'
        else:
            # To be filled later.
            return np.nan
    elif row['Time'] == '-':
        return '-'
    
    t = str(row['Time'])
        
    if row['Rnk'] == '1':
        return t
    else:
        return '+' + t[:int(len(t) / 2)]
    
    
    
results['Time'] = results.apply(clean_time, axis = 1)

In [49]:
# Nulls get same time as row above.
results['Time'].fillna(method = 'ffill', inplace = True)

In [50]:
# Check added times have at most 5 characters (no hours).
assert results.apply(lambda x: len(str(x['Time'])) > 6 and x['Rnk'] != '1', axis = 1).sum() == 0

In [51]:
results

,Rnk,GC,Timelag,BIB,Rider,Age,Team,Bonus,Time,type,year,stage
0,1,1.0,+0:00,41,MILLAR David,24,Cofidis,0,0:14:56,ITT,2001,1
1,2,2.0,+0:01,102,BOTERO Santiago,28,Kelme - Costa Blanca,0,+0:01,ITT,2001,1
2,3,3.0,+0:06,6,LEIPHEIMER Levi,27,US Postal Service,0,+0:06,ITT,2001,1
3,4,4.0,+0:17,155,GONZÁLEZ DE GALDEANO Igor,27,O.N.C.E. - Eroski,0,+0:17,ITT,2001,1
4,5,5.0,+0:23,152,BELOKI Joseba,28,O.N.C.E. - Eroski,0,+0:23,ITT,2001,1
...,...,...,...,...,...,...,...,...,...,...,...,...
75981,130,110.0,+4:33:12,156,HVIDEBERG Jonas Iversby,23,Team DSM,0,+4:44,Road Race,2022,21
75982,131,117.0,+4:48:07,162,CATALDO Dario,37,Trek - Segafredo,0,+4:44,Road Race,2022,21
75983,132,107.0,+4:23:20,183,JANSSENS Jimmy,33,Alpecin-Deceuninck,0,+4:46,Road Race,2022,21
75984,133,94.0,+3:59:57,212,AZPARREN Xabier Mikel,23,Euskaltel - Euskadi,0,+5:30,Road Race,2022,21


## Save Stages Data

In [52]:
info_df.to_csv('info', index = False)
ttt.to_csv('ttt', index = False)
results.to_csv('results', index = False)

## Scrape Final Classifications Data

In [53]:
base_url = 'https://www.procyclingstats.com/race/vuelta-a-espana/{}/{}'

# General, Points, and Mountain Classifications.
gc = pd.DataFrame()
points = pd.DataFrame()
kom = pd.DataFrame()

for year in range(2001, 2023):
    # General
    url = base_url.format(year, 'gc')
    tables = bs(fetch_url(url), 'html.parser').find_all('table', class_ = 'results basic moblist10')
    
    current_gc = pd.read_html(str(tables[1]), flavor = 'bs4', header = [0])[0]
    current_points = pd.read_html(str(tables[2]), flavor = 'bs4', header = [0])[0]
    current_kom = pd.read_html(str(tables[3]), flavor = 'bs4', header = [0])[0]
    
    for x in [current_gc, current_points, current_kom]:
        x['year'] = year
    
    gc = pd.concat([gc, current_gc], axis = 0)
    points = pd.concat([points, current_points], axis = 0)
    kom = pd.concat([kom, current_kom], axis = 0)   

In [54]:
gc.to_csv('gc_raw', index = False)
points.to_csv('points_raw', index = False)
kom.to_csv('kom_raw', index = False)

## Clean `gc`, `points`, and `kom`

In [55]:
gc.head()

,Rnk,Prev,▼▲,BIB,H2H,Rider,Age,Team,UCI,Pnt,Unnamed: 10,Time,Time won/lost,year
0,1,1.0,-,81,NaN,CASERO Ángel LuisFestina,29,Festina,NaN,400.0,NaN,70:49:05,..,2001
1,2,NaN,-,101,NaN,SEVILLA ÓscarKelme - Costa Blanca,25,Kelme - Costa Blanca,NaN,290.0,NaN,0:470:47,..,2001
2,3,NaN,-,6,NaN,LEIPHEIMER LeviUS Postal Service,27,US Postal Service,NaN,240.0,NaN,2:592:59,..,2001
3,4,NaN,-,1,NaN,HERAS RobertoUS Postal Service,27,US Postal Service,NaN,220.0,NaN,3:563:56,..,2001
4,5,NaN,-,29,NaN,MERCADO Juan MigueliBanesto.com,23,iBanesto.com,NaN,200.0,NaN,5:455:45,..,2001


In [56]:
gc.reset_index(drop = True, inplace = True)
gc.drop(['Prev', '▼▲', 'BIB', 'H2H', 'UCI', 'Pnt', 'Unnamed: 10', 'Time won/lost'], axis = 1, inplace = True)

In [57]:
gc[gc['Team'].isnull()]

,Rnk,Rider,Age,Team,Time,year
68,69,VÁZQUEZ José Manuel,26,NaN,1:47:141:47:14,2001
79,80,PEREZ CUAPIO Julio Alberto,24,NaN,2:02:072:02:07,2001
90,91,PEÑA Victor Hugo,27,NaN,2:17:512:17:51,2001
129,130,CARNEIRO SILVA Carlos Alberto,31,NaN,2:57:202:57:20,2001
233,95,REISS Andris,24,NaN,2:42:532:42:53,2002
260,122,GUERRA Alessandro,28,NaN,3:24:433:24:43,2002


In [58]:
# Same tactic as before.
gc['Rider'] = gc.apply(lambda x: str(x.Rider).removesuffix(str(x.Team)), axis = 1)

In [59]:
def clean_time_gc(row):
    if row['Time'].startswith(','):
        return np.nan
    elif row['Rnk'] == 1:
        return row['Time']
    else:
        t = row['Time']
        return '+' + t[:int(len(t) / 2)]
    
gc['Time'] = gc.apply(clean_time_gc, axis = 1)
gc['Time'].fillna(method = 'ffill', inplace = True)

In [60]:
gc

,Rnk,Rider,Age,Team,Time,year
0,1,CASERO Ángel Luis,29,Festina,70:49:05,2001
1,2,SEVILLA Óscar,25,Kelme - Costa Blanca,+0:47,2001
2,3,LEIPHEIMER Levi,27,US Postal Service,+2:59,2001
3,4,HERAS Roberto,27,US Postal Service,+3:56,2001
4,5,MERCADO Juan Miguel,23,iBanesto.com,+5:45,2001
...,...,...,...,...,...,...
3225,130,VAN DEN BERG Julius,25,EF Education-EasyPost,+5:15:43,2022
3226,131,OLIVEIRA Ivo,26,UAE Team Emirates,+5:19:55,2022
3227,132,MERLIER Tim,29,Alpecin-Deceuninck,+5:21:23,2022
3228,133,MAS Lluís,32,Movistar Team,+5:27:42,2022


In [61]:
points.head()

,Rnk,Prev,▼▲,BIB,H2H,Rider,Age,Team,UCI,Pnt,Points,Today,year
0,1,1.0,-,27,NaN,JIMÉNEZ José MaríaiBanesto.com,30,iBanesto.com,NaN,80.0,130.0,+130,2001
1,2,NaN,-,201,NaN,ZABEL ErikTeam Telekom,31,Team Telekom,NaN,20.0,125.0,+125,2001
2,3,NaN,-,6,NaN,LEIPHEIMER LeviUS Postal Service,27,US Postal Service,NaN,10.0,115.0,+115,2001
3,4,NaN,-,102,NaN,BOTERO SantiagoKelme - Costa Blanca,28,Kelme - Costa Blanca,NaN,NaN,102.0,+102,2001
4,5,NaN,-,101,NaN,SEVILLA ÓscarKelme - Costa Blanca,25,Kelme - Costa Blanca,NaN,NaN,101.0,+101,2001


In [62]:
points.reset_index(drop = True, inplace = True)
points.drop(['Prev', '▼▲', 'BIB', 'H2H', 'UCI', 'Pnt', 'Today'], axis = 1, inplace = True)

In [63]:
points['Rider'] = points.apply(lambda x: str(x.Rider).removesuffix(str(x.Team)), axis = 1)

In [64]:
# No value in entries with no points.
points = points.loc[points['Points'].notnull(), :]

In [65]:
kom.head()

,Rnk,Prev,▼▲,BIB,H2H,Rider,Age,Team,UCI,Pnt,Points,Today,year,#,Time
0,1.0,1.0,-,27.0,NaN,JIMÉNEZ José MaríaiBanesto.com,30.0,iBanesto.com,NaN,80.0,NaN,-,2001,NaN,NaN
1,2.0,NaN,-,147.0,NaN,MØLLER Claus MichaelMilaneza - MSS,32.0,Milaneza - MSS,NaN,20.0,NaN,-,2001,NaN,NaN
2,3.0,NaN,-,29.0,NaN,MERCADO Juan MigueliBanesto.com,23.0,iBanesto.com,NaN,10.0,NaN,-,2001,NaN,NaN
3,4.0,NaN,-,101.0,NaN,SEVILLA ÓscarKelme - Costa Blanca,25.0,Kelme - Costa Blanca,NaN,NaN,NaN,-,2001,NaN,NaN
4,5.0,NaN,-,2.0,NaN,RUBIERA José LuisUS Postal Service,28.0,US Postal Service,NaN,NaN,NaN,-,2001,NaN,NaN


In [66]:
kom.reset_index(drop = True, inplace = True)
kom.drop(['Prev', '▼▲', 'BIB', 'H2H', 'UCI', 'Pnt', 'Today', '#', 'Time'], axis = 1, inplace = True)

In [67]:
kom['Rider'] = kom.apply(lambda x: str(x.Rider).removesuffix(str(x.Team)), axis = 1)

In [68]:
kom = kom.loc[kom['Points'].notnull(), :]

## Save Classifications Data

In [69]:
gc.to_csv('gc', index = False)
points.to_csv('points', index = False)
kom.to_csv('kom', index = False)